# Create_metadata_for_epc_data

# setup

In [54]:
import csvw_functions
import csv
import json
import rdflib
import pandas as pd

## Certificates - create metadata file

In [55]:
# get file headings
with open('certificates.csv') as f: 
    csvreader=csv.reader(f)
    headings=next(iter(csvreader))
headings

['LMK_KEY',
 'ADDRESS1',
 'ADDRESS2',
 'ADDRESS3',
 'POSTCODE',
 'BUILDING_REFERENCE_NUMBER',
 'CURRENT_ENERGY_RATING',
 'POTENTIAL_ENERGY_RATING',
 'CURRENT_ENERGY_EFFICIENCY',
 'POTENTIAL_ENERGY_EFFICIENCY',
 'PROPERTY_TYPE',
 'BUILT_FORM',
 'INSPECTION_DATE',
 'LOCAL_AUTHORITY',
 'CONSTITUENCY',
 'COUNTY',
 'LODGEMENT_DATE',
 'TRANSACTION_TYPE',
 'ENVIRONMENT_IMPACT_CURRENT',
 'ENVIRONMENT_IMPACT_POTENTIAL',
 'ENERGY_CONSUMPTION_CURRENT',
 'ENERGY_CONSUMPTION_POTENTIAL',
 'CO2_EMISSIONS_CURRENT',
 'CO2_EMISS_CURR_PER_FLOOR_AREA',
 'CO2_EMISSIONS_POTENTIAL',
 'LIGHTING_COST_CURRENT',
 'LIGHTING_COST_POTENTIAL',
 'HEATING_COST_CURRENT',
 'HEATING_COST_POTENTIAL',
 'HOT_WATER_COST_CURRENT',
 'HOT_WATER_COST_POTENTIAL',
 'TOTAL_FLOOR_AREA',
 'ENERGY_TARIFF',
 'MAINS_GAS_FLAG',
 'FLOOR_LEVEL',
 'FLAT_TOP_STOREY',
 'FLAT_STOREY_COUNT',
 'MAIN_HEATING_CONTROLS',
 'MULTI_GLAZE_PROPORTION',
 'GLAZED_TYPE',
 'GLAZED_AREA',
 'EXTENSION_COUNT',
 'NUMBER_HABITABLE_ROOMS',
 'NUMBER_HEATED_ROOMS',

In [56]:
# get vocab
df_vocab=pd.read_csv('epc_vocab.csv',index_col='id',keep_default_na=False)
df_vocab

,label,comment,original_epc_datatype,csvw_datatype
id,,,,
original_epc_datatype,original_epc_datatype,"The datatype of a EPC variable, as given in th...",,
csvw_datatype,csvw_datatype,The CSVW datatype of a epc_vocab Property.,,
LMK_KEY,LMK_KEY,Individual lodgement identifier. Guaranteed to...,VARCHAR 64,string
ADDRESS1,ADDRESS1,First line of the address,VARCHAR 84,string
ADDRESS2,ADDRESS2,Second line of the address,VARCHAR 100,string
...,...,...,...,...
TENURE,TENURE,Describes the tenure type of the property. One...,VARCHAR 100,string
FIXED_LIGHTING_OUTLETS_COUNT,FIXED_LIGHTING_OUTLETS_COUNT,The number of fixed lighting outlets.,INT,integer
LOW_ENERGY_FIXED_LIGHT_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,The number of low-energy fixed lighting outlets.,INT,integer


In [58]:
metadata_schema_dict={
    "@context": "http://www.w3.org/ns/csvw",
    "@type": "Schema",
    'columns':[
    ]
}


for heading in headings:
    
    name=heading
    title=heading
    label=heading
    comment=df_vocab.loc[heading,'comment']
    datatype=df_vocab.loc[heading,'csvw_datatype']
    
    d={
        'name':name,
        'titles':title,
        'rdfs:label':label,
        'rdfs:comment':comment,
        'datatype':datatype,
        "propertyUrl": f"http://purl.org/berg/epc_vocab#{heading}",
        "aboutUrl": "http://purl.org/berg/epc_data#Certificate_{LMK_KEY}"
    }
    
    metadata_schema_dict['columns'].append(d)
    
# save the tableSchema object
with open('certificates-schema-metadata.json','w') as f:
    json.dump(metadata_schema_dict,f,indent=4)
    
metadata_schema_dict

{'@context': 'http://www.w3.org/ns/csvw',
 '@type': 'Schema',
 'columns': [{'name': 'LMK_KEY',
   'titles': 'LMK_KEY',
   'rdfs:label': 'LMK_KEY',
   'rdfs:comment': 'Individual lodgement identifier. Guaranteed to be unique and can be used to identify a certificate in the downloads and the API.',
   'datatype': 'string',
   'propertyUrl': 'http://purl.org/berg/epc_vocab#LMK_KEY',
   'aboutUrl': 'http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'},
  {'name': 'ADDRESS1',
   'titles': 'ADDRESS1',
   'rdfs:label': 'ADDRESS1',
   'rdfs:comment': 'First line of the address',
   'datatype': 'string',
   'propertyUrl': 'http://purl.org/berg/epc_vocab#ADDRESS1',
   'aboutUrl': 'http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'},
  {'name': 'ADDRESS2',
   'titles': 'ADDRESS2',
   'rdfs:label': 'ADDRESS2',
   'rdfs:comment': 'Second line of the address',
   'datatype': 'string',
   'propertyUrl': 'http://purl.org/berg/epc_vocab#ADDRESS2',
   'aboutUrl': 'http://purl.org/berg/epc_data#Certif

## Recommendations - create metadata file

## OLD

In [ ]:
# get embedded metadata from csv file
embedded_metadata_dict=csvw_functions.get_embedded_metadata('certificates.csv', relative_path=True)

# include additional information on columns
columns=embedded_metadata_dict['tableSchema']['columns']
for i in range(len(columns)):
    column=columns[i]
    aboutUrl='http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'
    heading, variable_string, description=get_glossary_info(column['name'])
    column['datatype']=get_datatype(variable_string)
    column['dc:description']=description
    column['propertyUrl']='http://purl.org/berg/epc_vocab#%s' % column['name']
    column['aboutUrl']=aboutUrl

# change some datatypes to fix errors in the glossary
def change_datatype(col_name,new_datatype):
    for x in embedded_metadata_dict['tableSchema']['columns']:
        if x['name']==col_name:
            x['datatype']=new_datatype
change_datatype('MULTI_GLAZE_PROPORTION','decimal')
change_datatype('EXTENSION_COUNT','decimal')
change_datatype('NUMBER_HABITABLE_ROOMS','decimal')
change_datatype('NUMBER_HEATED_ROOMS','decimal')
change_datatype('PHOTO_SUPPLY','decimal')

# create schema metadata dict
schema_metadata_dict=embedded_metadata_dict['tableSchema']
schema_metadata_dict["@context"]="http://www.w3.org/ns/csvw"
schema_metadata_dict["@type"]="Schema"

# save the tableSchema object
with open('certificates-schema-metadata.json','w') as f:
    json.dump(schema_metadata_dict,f,indent=4)
    
schema_metadata_dict

## functions

### get_glossary_info

In [16]:
with open('glossary_epc_domestic.txt') as f: glossary_text=f.read()

def get_glossary_info(col_name, st=glossary_text):
    ""
    a=st.find(col_name)
    b=st.find('\n',a+1)
    c=st.find('\n',b+1)
    
    variable_string=st[a+len(col_name):b].strip()
    
    description=st[b:c].strip()
    
    if not (variable_string.startswith('VARCHAR') 
            or variable_string.startswith('INT')
            or variable_string.startswith('DATE') 
            or variable_string.startswith('DECIMAL')
            or variable_string.startswith('DATETIME')
            or variable_string==''
           ):
    
        return get_glossary_info(col_name, st=st[a+len(col_name):])
    
    return col_name, variable_string, description

In [17]:
# test for single heading
get_glossary_info('LMK_KEY')

('LMK_KEY',
 'VARCHAR 64',
 'Individual lodgement identifier. Guaranteed to be unique and can be used to identify a certificate in the downloads and the API.')

In [18]:
# test for all headings
with open('certificates.csv') as f: 
    csvreader=csv.reader(f)
    headings=next(iter(csvreader))
for heading in headings:
    print(get_glossary_info(heading))

('LMK_KEY', 'VARCHAR 64', 'Individual lodgement identifier. Guaranteed to be unique and can be used to identify a certificate in the downloads and the API.')
('ADDRESS1', 'VARCHAR 84', 'First line of the address')
('ADDRESS2', 'VARCHAR 100', 'Second line of the address')
('ADDRESS3', 'VARCHAR 100', 'Third line of the address')
('POSTCODE', 'VARCHAR 8', 'The postcode of the property')
('BUILDING_REFERENCE_NUMBER', 'VARCHAR 12', 'Unique identifier for the property.')
('CURRENT_ENERGY_RATING', 'VARCHAR 8', "Current energy rating converted into a linear 'A to G' rating (where A is the most energy efficient and G is the least energy efficient)")
('POTENTIAL_ENERGY_RATING', 'VARCHAR 8', "Estimated potential energy rating converted into a linear 'A to G' rating (where A is the most energy efficient and G is the least energy efficient)")
('CURRENT_ENERGY_EFFICIENCY', 'INT', 'Based on cost of energy, i.e. energy required for space heating, water heating and lighting [in kWh/year] multiplied by 

### get_datatype

In [19]:
def get_datatype(variable_string):
    ""
    if variable_string=='':
        return 'string'
    elif variable_string.startswith('VARCHAR'):
        return 'string'
    elif variable_string.startswith('INT'):
        return 'integer'
    elif variable_string.startswith('DECIMAL'):
        return 'decimal'
    elif variable_string.startswith('DATETIME'):
        return {'base':'datetime','format':'yyyy-MM-dd HH:mm:ss'}
    elif variable_string.startswith('DATE'):
        return 'date'
    else:
        raise Exception

In [20]:
# check for all headings
for heading in headings:
    _, variable_string, description=get_glossary_info(heading)
    print(heading, get_datatype(variable_string))

LMK_KEY string
ADDRESS1 string
ADDRESS2 string
ADDRESS3 string
POSTCODE string
BUILDING_REFERENCE_NUMBER string
CURRENT_ENERGY_RATING string
POTENTIAL_ENERGY_RATING string
CURRENT_ENERGY_EFFICIENCY integer
POTENTIAL_ENERGY_EFFICIENCY integer
PROPERTY_TYPE string
BUILT_FORM string
INSPECTION_DATE date
LOCAL_AUTHORITY string
CONSTITUENCY string
COUNTY string
LODGEMENT_DATE date
TRANSACTION_TYPE string
ENVIRONMENT_IMPACT_CURRENT integer
ENVIRONMENT_IMPACT_POTENTIAL integer
ENERGY_CONSUMPTION_CURRENT integer
ENERGY_CONSUMPTION_POTENTIAL integer
CO2_EMISSIONS_CURRENT decimal
CO2_EMISS_CURR_PER_FLOOR_AREA decimal
CO2_EMISSIONS_POTENTIAL decimal
LIGHTING_COST_CURRENT integer
LIGHTING_COST_POTENTIAL integer
HEATING_COST_CURRENT integer
HEATING_COST_POTENTIAL integer
HOT_WATER_COST_CURRENT integer
HOT_WATER_COST_POTENTIAL integer
TOTAL_FLOOR_AREA decimal
ENERGY_TARIFF string
MAINS_GAS_FLAG string
FLOOR_LEVEL string
FLAT_TOP_STOREY string
FLAT_STOREY_COUNT integer
MAIN_HEATING_CONTROLS string
MU

## certificates

In [25]:
# get embedded metadata from csv file
embedded_metadata_dict=csvw_functions.get_embedded_metadata('certificates.csv', relative_path=True)

# include additional information on columns
columns=embedded_metadata_dict['tableSchema']['columns']
for i in range(len(columns)):
    column=columns[i]
    aboutUrl='http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'
    heading, variable_string, description=get_glossary_info(column['name'])
    column['datatype']=get_datatype(variable_string)
    column['dc:description']=description
    column['propertyUrl']='http://purl.org/berg/epc_vocab#%s' % column['name']
    column['aboutUrl']=aboutUrl

# change some datatypes to fix errors in the glossary
def change_datatype(col_name,new_datatype):
    for x in embedded_metadata_dict['tableSchema']['columns']:
        if x['name']==col_name:
            x['datatype']=new_datatype
change_datatype('MULTI_GLAZE_PROPORTION','decimal')
change_datatype('EXTENSION_COUNT','decimal')
change_datatype('NUMBER_HABITABLE_ROOMS','decimal')
change_datatype('NUMBER_HEATED_ROOMS','decimal')
change_datatype('PHOTO_SUPPLY','decimal')

# create schema metadata dict
schema_metadata_dict=embedded_metadata_dict['tableSchema']
schema_metadata_dict["@context"]="http://www.w3.org/ns/csvw"
schema_metadata_dict["@type"]="Schema"

# save the tableSchema object
with open('certificates-schema-metadata.json','w') as f:
    json.dump(schema_metadata_dict,f,indent=4)
    
schema_metadata_dict

{'columns': [{'titles': {'und': ['LMK_KEY']},
   'name': 'LMK_KEY',
   'datatype': 'string',
   'dc:description': 'Individual lodgement identifier. Guaranteed to be unique and can be used to identify a certificate in the downloads and the API.',
   'propertyUrl': 'http://purl.org/berg/epc_vocab#LMK_KEY',
   'aboutUrl': 'http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'},
  {'titles': {'und': ['ADDRESS1']},
   'name': 'ADDRESS1',
   'datatype': 'string',
   'dc:description': 'First line of the address',
   'propertyUrl': 'http://purl.org/berg/epc_vocab#ADDRESS1',
   'aboutUrl': 'http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'},
  {'titles': {'und': ['ADDRESS2']},
   'name': 'ADDRESS2',
   'datatype': 'string',
   'dc:description': 'Second line of the address',
   'propertyUrl': 'http://purl.org/berg/epc_vocab#ADDRESS2',
   'aboutUrl': 'http://purl.org/berg/epc_data#Certificate_{LMK_KEY}'},
  {'titles': {'und': ['ADDRESS3']},
   'name': 'ADDRESS3',
   'datatype': 'string',
   'dc